In [1]:
import os

!pwd
os.chdir("../")
!pwd

/home/tensorthiru/ml_lifecycle_sample/research
/home/tensorthiru/ml_lifecycle_sample


In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_file: Path
    status_file: Path

In [15]:
from src.pipeline_sample.utils.common import *
from src.pipeline_sample.constants import *

In [17]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 schema_file_path: Path = SCHEMA_FILE_PATH,
                 params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        if type(self.config.artifacts_root) == str:
            create_directories([self.config.artifacts_root])
        else:
            create_directories(self.config.artifacts_root)
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        
        status_file_path = self.config.data_validation.status_file
        config = self.config.data_transformation
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_file=Path(config.data_file),
            status_file=Path(status_file_path)
        )
        
        return data_transformation_config

In [18]:
import os
from src.pipeline_sample import logger

In [20]:
##Component
import json
import pandas as pd

from sklearn.model_selection import train_test_split

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        create_directories([self.config.root_dir])
    
    def initiate_data_transformation(self):
        
        status_check = json.load(open(self.config.status_file, 'r'))
        if status_check['validation_status'] == "Success":
            data = pd.read_csv(self.config.data_file)
            
            ##Data Transformation
            train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
            logger.info(f"Dataset Split into Train and Test with shapes: {train_data.shape} and {test_data.shape} respectively.")
            
            train_data.to_csv(os.path.join(self.config.root_dir, "train_data.csv"), index=False)
            test_data.to_csv(os.path.join(self.config.root_dir, "test_data.csv"), index=False)